In [1]:
import random
import matplotlib.pyplot as plt
import cv2

from detectron2.data import DatasetCatalog, MetadataCatalog, get_detection_dataset_dicts, print_instances_class_histogram
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.utils.logger import setup_logger

from data_utils import read_split_file, register_dataset

setup_logger()
%matplotlib inline

In [2]:
mixed_sets = read_split_file("data/panels/mixed/split.txt")

# Register mixed datasets
for spl, im_paths in zip(["train", "val", "test"], mixed_sets):
    DatasetCatalog.register(
        f"{spl}", lambda im_paths=im_paths: register_dataset(im_paths)
    )
    MetadataCatalog.get(f"mixed_{spl}").set(
        thing_classes=["label", "button"],
        thing_colors=[(0, 255, 0), (0, 0, 255)],
    )

In [3]:
trainset = get_detection_dataset_dicts("train", filter_empty=False)
valset = get_detection_dataset_dicts("val", filter_empty=False)
testset = get_detection_dataset_dicts("test", filter_empty=False)
dataset = trainset + valset + testset
print(len(dataset))

registering mixed dataset: 100%|██████████| 22/22 [00:00<00:00, 26.60it/s]

108


In [5]:
print_instances_class_histogram(testset, ["label", "button"])

[02/28 21:08:31 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   label    | 423          |   button   | 423          |
|            |              |            |              |
|   total    | 846          |            |              |


In [ ]:
sampled_dicts = random.sample(trainset, 20)

d = sampled_dicts[0]
og_img = cv2.imread(d["file_name"])
plt.imshow(og_img[:, :, ::-1])

In [ ]:
def get_optimal_font_scale(text, width):

    for scale in reversed(range(0, 60, 1)):
        textSize = cv2.getTextSize(text, fontFace=cv2.FONT_HERSHEY_DUPLEX, fontScale=scale/10, thickness=1)
        new_width = textSize[0][0]
        if (new_width <= width):
            return scale/10
    return 1

In [ ]:
import json
import os

from data_utils import generate_bbox, generate_gt_mask_coords

symbol_map = {"<|>": "open", ">|<": "close", "^": "alarm", "&": "call", "#": "stop"}

annos = json.load(open("data/panels/mixed/annotations.json"))
for d in sampled_dicts:
    print(d["file_name"])
    img = cv2.imread(d["file_name"])
    height, width = img.shape[:2]
    visualizer = Visualizer(
        img[:, :, ::-1],
        metadata=MetadataCatalog.get("mixed_train"),
        scale=1,
        instance_mode=ColorMode.SEGMENTATION,
    )
    out = visualizer.draw_dataset_dict(d)
    out = out.get_image()
    img_dict = annos[os.path.basename(d["file_name"])]
    for r in img_dict["regions"]:
        pair = r["region_attributes"].get("pair")
        if pair is None or pair == "":
            continue
        
        pair = pair.rstrip().lower()
        if r["region_attributes"]["category_id"] == "button" and (r["region_attributes"]["pair"] is not None or r["region_attributes"]["pair"] != ""):
            bbox = generate_bbox(*generate_gt_mask_coords(r, height, width), height, width)
            cv2.putText(
                out,
                text=pair if pair not in symbol_map else symbol_map[pair],
                org=(int(bbox[0]) + 5, int((bbox[1] + bbox[3]) / 2)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(255, 0, 0),
                thickness=2,
            )

    cv2.imwrite(f"big_graphic/{os.path.basename(d['file_name'])}", out[:, :, ::-1])
    # plt.imshow(out.get_image())
    # plt.axis("off")
    # plt.show()